In [1]:
import pandas as pd 
import numpy as np

Use read_html function to scrape the table from Wikipedia page.

In [2]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

There are 2 tables on the page, so I make sure that I select the correct object scraped [0]

In [3]:
df1 = df[0]
df1.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Check how many rows we have. We can see that some values for Borough are "Not Assigned" and we need to delete them.

In [4]:
df1.shape

(180, 3)

Deleting "Not assigned" rows

In [5]:
df1.drop(df1[df1['Borough'] == 'Not assigned'].index, inplace=True)
df1.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Checking if there are any NaN (missing data) values

In [7]:
missing_data = df1.isnull()
for column in missing_data.columns.values.tolist():
    print(column)
    print(missing_data[column].value_counts())
    print('')

Postal Code
False    103
Name: Postal Code, dtype: int64

Borough
False    103
Name: Borough, dtype: int64

Neighborhood
False    103
Name: Neighborhood, dtype: int64



There are no NaN values in Neighborhood column, meaning that the table is complete!

In [8]:
! pip install geocoder

     |████████████████████████████████| 102kB 7.5MB/s ta 0:00:011


In [9]:
import requests
import geocoder

lat_lng_coords = None

postal_code = 'M5G'

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

KeyboardInterrupt: 

In [10]:
print(g)

<[REQUEST_DENIED] Google - Geocode [empty]>


In [11]:
g = geocoder.google('Mountain View, CA')

In [12]:
g

<[REQUEST_DENIED] Google - Geocode [empty]>

Not working, so we will try using read_csv function (try later).

In [13]:
df_coord = pd.read_csv("http://cocl.us/Geospatial_data")

In [29]:
df_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Now we need to merge df and df_coord using Left join, where df1 is Left and on "postal code column, so that for each postal code in dataframe df, we will get latitude and longitude coordinates

In [26]:
df_combined = df1.merge(df_coord, on = 'Postal Code', how = 'left')

In [30]:
df_combined.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [35]:
df_combined.dtypes

Postal Code      object
Borough          object
Neighborhood     object
Latitude        float64
Longitude       float64
dtype: object

In [36]:
missing_data2 = df_combined.isnull()
for column in missing_data2.columns.values.tolist():
    print(column)
    print(missing_data2[column].value_counts())
    print('')

Postal Code
False    103
Name: Postal Code, dtype: int64

Borough
False    103
Name: Borough, dtype: int64

Neighborhood
False    103
Name: Neighborhood, dtype: int64

Latitude
False    103
Name: Latitude, dtype: int64

Longitude
False    103
Name: Longitude, dtype: int64



So there are no NaN values and table looks complete!